In [1]:
import numpy as np
import torch

from s3ts.helper_functions import *

In [2]:
dm = load_tsdataset("WISDM", "./datasets/", 128, 8, 48, 1, True, None, [30, 31, 32, 33, 34, 35], True, 1)

Loaded dataset WISDM with a total of 1096480 observations for window size 48
Using 443636 observations for training and 154375 observations for validation and test


In [3]:
from s3ts.api.nets.methods import create_model_from_DM, train_model

model = create_model_from_DM(dm, "ts", "cnn", "mlp", "cls", "test", 20, 32, 2, 0.001)

Input shape:  torch.Size([1, 3, 48])
Latent shape:  torch.Size([1, 80, 12])
[tensor(960), 32, 32, 6]


In [4]:
train_model(dm, model, 5, {
    "default_root_dir": "training",
    "accelerator": "gpu",
    "seed": 42
})

Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name       | Type                 | Params
-----------------------------------------------------
0  | encoder    | CNN_TS               | 12.9 K
1  | decoder    | MultiLayerPerceptron | 32.0 K
2  | flatten    | Flatten              | 0     
3  | softmax    | Softmax              | 0     
4  | train_acc  | MulticlassAccuracy   | 0     
5  | train_f1   | MulticlassF1Score    | 0     
6  | val_acc    | MulticlassA

Epoch 4: 100%|██████████| 3466/3466 [00:45<00:00, 76.46it/s, v_num=24, train_loss_step=0.200, val_loss=0.707, val_acc=0.756, val_auroc=0.960, train_loss_epoch=0.252, train_acc=0.922]  

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 3466/3466 [00:45<00:00, 76.46it/s, v_num=24, train_loss_step=0.200, val_loss=0.707, val_acc=0.756, val_auroc=0.960, train_loss_epoch=0.252, train_acc=0.922]
Input shape:  torch.Size([1, 3, 48])
Latent shape:  torch.Size([1, 80, 12])
[tensor(960), 32, 32, 6]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation DataLoader 0: 100%|██████████| 1207/1207 [00:16<00:00, 74.33it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │     0.820956826210022     │
│         val_auroc         │    0.9649650454521179     │
│          val_f1           │    0.7980151176452637     │
│         val_loss          │    0.6010432243347168     │
└───────────────────────────┴───────────────────────────┘

(WrapperModel(
   (encoder): CNN_TS(
     (cnn_0): Sequential(
       (0): Conv1d(3, 10, kernel_size=(3,), stride=(1,), padding=same)
       (1): ReLU()
       (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
     )
     (cnn_1): Sequential(
       (0): Conv1d(10, 20, kernel_size=(3,), stride=(1,), padding=same)
       (1): ReLU()
       (2): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
       (3): Dropout(p=0.35, inplace=False)
     )
     (cnn_2): Sequential(
       (0): Conv1d(20, 40, kernel_size=(3,), stride=(1,), padding=same)
       (1): BatchNorm1d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU()
       (3): Dropout(p=0.4, inplace=False)
     )
     (cnn_3): Conv1d(40, 80, kernel_size=(3,), stride=(1,), padding=same)
   )
   (decoder): MultiLayerPerceptron(
     (fcn_layer_0): Linear(in_features=960, out_features=32, bias=True)
     (act_layer_0): ReLU()
     (fcn_layer_1): Linear(in_features=32, out_

In [7]:
import torchmetrics as tm

In [8]:
auroc_online = tm.AUROC(task="multiclass", num_classes=6)
dataloader = dm.val_dataloader()

In [9]:
auroc_batch = []

In [10]:
model.eval()
auroc_online.reset()
for batch_idx, data in enumerate(dataloader):
    preds = model.logits(data["series"])
    auroc_online(preds, data["label"])

In [11]:
auroc_online.compute()

tensor(0.9598)